In [13]:
import subprocess
import pickle
import datetime

In [14]:
def parse_noxim_output(output):
    """
    Parse the Noxim simulation output and extract relevant metrics.
    """
    metrics = {}
    buffer_data = []
    lines = output.splitlines()
    
    for line in lines:
        if "Total received packets" in line:
            metrics["total_received_packets"] = int(line.split(":")[-1].strip())
        elif "Total received flits" in line:
            metrics["total_received_flits"] = int(line.split(":")[-1].strip())
        elif "Received/Ideal flits Ratio" in line:
            metrics["received_ideal_flits_ratio"] = float(line.split(":")[-1].strip())
        elif "Average wireless utilization" in line:
            metrics["average_wireless_utilization"] = float(line.split(":")[-1].strip())
        elif "Global average delay (cycles)" in line:
            metrics["global_average_delay"] = float(line.split(":")[-1].strip())
        elif "Max delay (cycles)" in line:
            metrics["max_delay"] = int(line.split(":")[-1].strip())
        elif "Network throughput (flits/cycle)" in line:
            metrics["network_throughput"] = float(line.split(":")[-1].strip())
        elif "Average IP throughput (flits/cycle/IP)" in line:
            metrics["average_ip_throughput"] = float(line.split(":")[-1].strip())
        elif "Total energy (J)" in line:
            metrics["total_energy"] = float(line.split(":")[-1].strip())
        elif "Dynamic energy (J)" in line:
            metrics["dynamic_energy"] = float(line.split(":")[-1].strip())
        elif "Static energy (J)" in line:
            metrics["static_energy"] = float(line.split(":")[-1].strip())
        elif line.startswith("Router id"):
            # Start of buffer data
            buffer_start_index = lines.index(line) + 2
            for buffer_line in lines[buffer_start_index:]:
                if buffer_line.strip() == "":
                    break
                # Parse the tab-separated values into a structured dictionary
                buffer_values = buffer_line.split("\t")
                router_data = {
                    "router_id": int(buffer_values[0]),
                    "buffer_n_mean": float(buffer_values[1]) if buffer_values[1] else None,
                    "buffer_n_max": int(buffer_values[2]) if buffer_values[2] else None,
                    "buffer_e_mean": float(buffer_values[3]) if buffer_values[3] else None,
                    "buffer_e_max": int(buffer_values[4]) if buffer_values[4] else None,
                    "buffer_s_mean": float(buffer_values[5]) if buffer_values[5] else None,
                    "buffer_s_max": int(buffer_values[6]) if buffer_values[6] else None,
                    "buffer_w_mean": float(buffer_values[7]) if buffer_values[7] else None,
                    "buffer_w_max": int(buffer_values[8]) if buffer_values[8] else None,
                    "buffer_l_mean": float(buffer_values[9]) if buffer_values[9] else None,
                    "buffer_l_max": int(buffer_values[10]) if buffer_values[10] else None,
                }
                buffer_data.append(router_data)
    
    metrics["buffer_data"] = buffer_data
    return metrics

In [20]:
import os

def save_to_pickle(data, filename):
    """
    Save the parsed data to a pickle file.
    """
    # Ensure the directory exists
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
    print(f"Data saved to {filename}")

In [ ]:
FILENAME = "DA+BMAC"

# Create a dictionary to store results for all PIRs
all_results = {}

# Generate a timestamp for the run
run_timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
output_pickle_file = f"./{FILENAME}/{FILENAME}_results_{run_timestamp}.pkl"

for j in range(30):
    i = round(0.001 + (0.003 * j), 4)
    print(f"Running simulation with PIR={i}")

    result = subprocess.run(['./noxim', '-config', f'../custom_config/{FILENAME}.yaml', '-pir', str(i), 'poisson'],
                            capture_output=True, text=True, cwd='../bin')

    output = result.stdout
    metrics = parse_noxim_output(output)
    all_results[i] = metrics

# Save all results to a single pickle file
save_to_pickle(all_results, output_pickle_file)

Running simulation with PIR=0.001
Running simulation with PIR=0.004
Running simulation with PIR=0.004
Running simulation with PIR=0.007
Running simulation with PIR=0.007
Running simulation with PIR=0.01
Running simulation with PIR=0.01
Running simulation with PIR=0.013
Running simulation with PIR=0.013
Running simulation with PIR=0.016
Running simulation with PIR=0.016
Running simulation with PIR=0.019
Running simulation with PIR=0.019
Running simulation with PIR=0.022
Running simulation with PIR=0.022
Running simulation with PIR=0.025
Running simulation with PIR=0.025
Running simulation with PIR=0.028
Running simulation with PIR=0.028
Running simulation with PIR=0.031
Running simulation with PIR=0.031
Running simulation with PIR=0.034
Running simulation with PIR=0.034
Running simulation with PIR=0.037
Running simulation with PIR=0.037
Running simulation with PIR=0.04
Running simulation with PIR=0.04
Running simulation with PIR=0.043
Running simulation with PIR=0.043
Running simulation

FileNotFoundError: [Errno 2] No such file or directory: './DA-BMAC/DA-BMAC_results_20250902_184033.pkl'